In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split, cross_val_predict, cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.svm import LinearSVC
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, mutual_info_classif
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
import nltk
from nltk.tokenize import word_tokenize

## Combining chunks of the PAN16 dataset

In [3]:
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test4.csv')

In [4]:
df2 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test5.csv')

In [5]:
df3 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test6.csv')

In [6]:
df4 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test7.csv')

In [7]:
df5 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test8.csv')

In [8]:
df6 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test9.csv')

In [9]:
df7 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test10.csv')

In [10]:
df8 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test11.csv')

In [11]:
dataset = pd.DataFrame(columns=['id', 'tweet', 'age'])
dataset = df.append([df, df2, df3, df4, df5, df6, df7, df8], ignore_index = True)
print (len(dataset))
print(dataset.tail())

105086
        Unnamed: 0                                id  \
105081       20770  54db3923f033e1dd6a82222aa2604ab9   
105082       20771  54db3923f033e1dd6a82222aa2604ab9   
105083       20772  54db3923f033e1dd6a82222aa2604ab9   
105084       20773  54db3923f033e1dd6a82222aa2604ab9   
105085       20774  54db3923f033e1dd6a82222aa2604ab9   

                                                    tweet    age  
105081  "This is cool! RT @bikewa: They're all the buz...  25-34  
105082          "@bikewa absolutely the best. Very cool."  25-34  
105083  "Pinterest quietly pops out a fantastic new la...  25-34  
105084  "Photo: Inhaled! #fitfoodie http://t.co/2RE6a4...  25-34  
105085  "I'm watching Vikings (150 others checked-in) ...  25-34  


In [14]:
#dataset = dataset.dropna()
dataset = dataset.dropna()

In [15]:
print (len(dataset))

93707


In [16]:
print (dataset.tail())

        Unnamed: 0                                id  \
105081       20770  54db3923f033e1dd6a82222aa2604ab9   
105082       20771  54db3923f033e1dd6a82222aa2604ab9   
105083       20772  54db3923f033e1dd6a82222aa2604ab9   
105084       20773  54db3923f033e1dd6a82222aa2604ab9   
105085       20774  54db3923f033e1dd6a82222aa2604ab9   

                                                    tweet    age  
105081  "This is cool! RT @bikewa: They're all the buz...  25-34  
105082          "@bikewa absolutely the best. Very cool."  25-34  
105083  "Pinterest quietly pops out a fantastic new la...  25-34  
105084  "Photo: Inhaled! #fitfoodie http://t.co/2RE6a4...  25-34  
105085  "I'm watching Vikings (150 others checked-in) ...  25-34  


In [12]:
print (dataset['id'].value_counts())

00db29c2dc1d87c8f07b72d753f7f2c0    2000
025caf45aaec8db680b23464b930e8bc    1632
01af1cc8a0aad1ba7d237d1144338bc6    1558
43d7f732e9da60bc1d8f13294c3a3f1c    1000
3441a01d2d0c4dfe72216b104a1eba95    1000
                                    ... 
278578135bfa56ada28f14085db9307b      32
40b8af7721fd105d0f5ecb5dad74a991      32
07011b7672dfa64896c4c7b1b41ef95d      30
515618c30574293240c3b346bdf04b5f      30
19a9a1af56559b5d299fae4cbac79a0f      18
Name: id, Length: 161, dtype: int64


In [13]:
dataset['age'].value_counts()

35-49    46107
25-34    35849
50-64    15522
18-24     6182
65-xx     1426
Name: age, dtype: int64

In [15]:
print(dataset['id'].to_string())

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



## Language detection: removing non-english tweets
Store them in a different variable!  
Including tweets in other languages might be interesting

In [54]:
df1 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test4.csv')
df2 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test5.csv')
df3 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test6.csv')
df4 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test7.csv')
df5 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test8.csv')
df6 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test9.csv')
df7 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test10.csv')
df8 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test11.csv')
df9 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test12.csv')
df10 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test13.csv')
df11 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test14.csv')

df = df1.append([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11], ignore_index = True)
df = df.dropna()

df['id'].value_counts()

80d92cf1ea27e001428c05bed7de7aae    1000
3bb08fb1b8b3d0d35bd70e1753840d2c    1000
0394cf313b58ea16e3776dab47042e5f    1000
5c83a98f3d8627a98496d9bbcbb97c3f    1000
2b008cd4452e21f9e8ca50417c27de2a    1000
                                    ... 
40b8af7721fd105d0f5ecb5dad74a991      32
07011b7672dfa64896c4c7b1b41ef95d      30
515618c30574293240c3b346bdf04b5f      30
5a462f6ae94aa6db98cc3998694cf051      30
19a9a1af56559b5d299fae4cbac79a0f      18
Name: id, Length: 233, dtype: int64

## Parsing

In [2]:
import spacy
from spacy_langdetect import LanguageDetector

model = spacy.load("en")

OSError: [E050] Can't find model 'en'. It doesn't seem to be a shortcut link, a Python package or a valid path to a data directory.

In [57]:
parsed = []
for i in df['tweet']:
    parsed.append(model(i))
parsed

["@zulahni Oh, honey. Don't force us to stage an intervention. Look at your life. Look at your choices. #sassygayfriend",
 "@BrianEnigma Re: ARG Tools 2.1: bad news, the HTML entity is actually "&amp;mdash" (no "e"). #htmlisweird",
 "@BrianEnigma Confirmed. It's like magic! :)",
 "College flashback: I just backed New Studio Album from Toad the Wet Sprocket! on @Kickstarter http://t.co/tqZFyzahhO",
 "If you're wondering why I'm not tweeting so much this weekend, check my other account: @curtiscchen #wwcon13",
 "#wwcon13 day 2 (@ Sheraton Portland Airport Hotel - @spg) http://t.co/DbCvIoUutV",
 ".@ninjahq "Scorekeeper XL," currently FREE on app store: https://t.co/Cv9zHfkJon",
 ""You call this archaeology?" #wwcon13 #nonsequitur http://t.co/o8g389UpUj",
 "I'll take Manhattan #wwcon13 #nobittersthough http://t.co/bGdlaWW0Gv",
 "Maybe it's just me, but emergency vehicles driving up &amp; down the runway shooting off flares seems bad. #pdx",
 "Last day of #wwcon13! (@ Sheraton Portland Airp

In [85]:
print(len(df['tweet']))

142848


In [94]:
idx = 0
pos_tags= []

for i in df['tweet']:
    pos_tags.append([(x.pos_) for x in parsed[idx]])
    idx +=1
pos_tags

[['PUNCT',
  'PUNCT',
  'INTJ',
  'PUNCT',
  'NOUN',
  'PUNCT',
  'AUX',
  'PART',
  'VERB',
  'PRON',
  'PART',
  'VERB',
  'DET',
  'NOUN',
  'PUNCT',
  'VERB',
  'ADP',
  'DET',
  'NOUN',
  'PUNCT',
  'VERB',
  'ADP',
  'DET',
  'NOUN',
  'PUNCT',
  'NOUN',
  'NOUN',
  'PUNCT'],
 ['PUNCT',
  'PROPN',
  'ADP',
  'PUNCT',
  'PROPN',
  'NOUN',
  'NUM',
  'PUNCT',
  'ADJ',
  'NOUN',
  'PUNCT',
  'DET',
  'NOUN',
  'NOUN',
  'AUX',
  'ADV',
  'PUNCT',
  'CCONJ',
  'PROPN',
  'PUNCT',
  'PUNCT',
  'INTJ',
  'PUNCT',
  'NOUN',
  'PUNCT',
  'PUNCT',
  'PUNCT',
  'NOUN',
  'NOUN',
  'PUNCT'],
 ['PUNCT',
  'PROPN',
  'VERB',
  'PUNCT',
  'PRON',
  'AUX',
  'INTJ',
  'NOUN',
  'PUNCT',
  'X',
  'PUNCT'],
 ['PUNCT',
  'NOUN',
  'NOUN',
  'PUNCT',
  'PRON',
  'ADV',
  'VERB',
  'PROPN',
  'PROPN',
  'PROPN',
  'ADP',
  'NOUN',
  'DET',
  'PROPN',
  'PROPN',
  'PUNCT',
  'ADP',
  'PROPN',
  'NOUN',
  'PUNCT'],
 ['PUNCT',
  'SCONJ',
  'PRON',
  'AUX',
  'VERB',
  'ADV',
  'PRON',
  'AUX',
  'PART'

In [ ]:
#punct = double quotation marks. Delete??

In [ ]:
# take small subset to try a few things out

In [21]:
# Loading the dataset
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/celebrities.csv')
df = df.rename(columns={"text": "tweet"})
df = df.drop('Unnamed: 0', 1)
df = df.groupby('id').agg(lambda x: x.tolist())
df['tweet'] = df['tweet'].str.join(sep='')
df ['tweet'] = df['tweet'].replace(r'\t',  ' ', regex=True)

In [23]:
# Loading the dataset
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/PAN15train.csv')
df = df.rename(columns={"text": "tweet"})
df = df.drop('Unnamed: 0', 1)
df = df.groupby('id').agg(lambda x: x.tolist())
df['tweet'] = df['tweet'].str.join(sep='')
df ['tweet'] = df['tweet'].replace(r'\t',  ' ', regex=True)
for idx, age_cat in enumerate(df['age']):
    df['age'][idx] = df['age'][idx][0]


## PAN16

In [19]:
df1 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test4.csv')
df2 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test5.csv')
df3 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test6.csv')
df4 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test7.csv')
df5 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test8.csv')
df6 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test9.csv')
df7 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test10.csv')
df8 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test11.csv')
df9 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test12.csv')
df10 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test13.csv')
df11 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test14.csv')
df12 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test15.csv')
df13 = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/test16.csv')

df = df1.append([df2, df3, df4, df5, df6, df7, df8, df9, df10, df11], ignore_index = True)
df = df.dropna()

df['id'].value_counts()

80d92cf1ea27e001428c05bed7de7aae    1000
2b008cd4452e21f9e8ca50417c27de2a    1000
1615a8f53c2f674adbec6829fa99e52e    1000
419453e20c2a73147960a6602d014b46    1000
611d21bbf8ba18cfedb76104c117ff76    1000
                                    ... 
40b8af7721fd105d0f5ecb5dad74a991      32
515618c30574293240c3b346bdf04b5f      30
5a462f6ae94aa6db98cc3998694cf051      30
07011b7672dfa64896c4c7b1b41ef95d      30
19a9a1af56559b5d299fae4cbac79a0f      18
Name: id, Length: 233, dtype: int64

In [ ]:
#  most frequent features: 
def get_top_tf_idf_words(response, top_n=2):
    sorted_nzs = np.argsort(response.data)[:-(top_n+1):-1]
    return feature_names[response.indices[sorted_nzs]]

In [13]:
# grid search
X = df['tweet']
y = df['age']

tfidf = TfidfVectorizer(analyzer="char", ngram_range=(1,1))
logreg = LogisticRegression()

estimator = Pipeline(steps=[('vectorizer', tfidf), ('classifier', logreg)])

param_grid = {
    "vectorizer__analyzer": ["char", "word"],
    "vectorizer__max_features":[39000, 40000, 41000, 42000],
    "vectorizer__ngram_range": [(1,1), (1,2), (3,3)],
    "classifier__C": [1, 10]
}

search = GridSearchCV(
    estimator=estimator,
    param_grid=param_grid, 
    scoring="f1_micro",
    cv=10,
    verbose=10,
    return_train_score=True,
    n_jobs=3)

In [14]:
search.fit(X, y)

Fitting 10 folds for each of 48 candidates, totalling 480 fits


[Parallel(n_jobs=3)]: Using backend LokyBackend with 3 concurrent workers.
[Parallel(n_jobs=3)]: Done   2 tasks      | elapsed:   38.6s
[Parallel(n_jobs=3)]: Done   7 tasks      | elapsed:  1.9min
[Parallel(n_jobs=3)]: Done  12 tasks      | elapsed:  3.6min
[Parallel(n_jobs=3)]: Done  19 tasks      | elapsed:  7.1min
[Parallel(n_jobs=3)]: Done  26 tasks      | elapsed: 10.0min
[Parallel(n_jobs=3)]: Done  35 tasks      | elapsed: 12.0min
[Parallel(n_jobs=3)]: Done  44 tasks      | elapsed: 15.0min
[Parallel(n_jobs=3)]: Done  55 tasks      | elapsed: 20.4min
[Parallel(n_jobs=3)]: Done  66 tasks      | elapsed: 24.4min
D:\Anaconda\lib\site-packages\joblib\externals\loky\process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning
[Parallel(n_jobs=3)]: Done  79 tasks      | elapsed: 30.0min
[Parallel(n_jobs=3)]: Done  92 tasks      | elap

GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=Pipeline(memory=None,
                                steps=[('vectorizer',
                                        TfidfVectorizer(analyzer='char',
                                                        binary=False,
                                                        decode_error='strict',
                                                        dtype=<class 'numpy.float64'>,
                                                        encoding='utf-8',
                                                        input='content',
                                                        lowercase=True,
                                                        max_df=1.0,
                                                        max_features=None,
                                                        min_df=1,
                                                        ngram_range=(1, 1),
                                   

In [15]:
search.best_score_

0.45744427643369173

In [52]:
#base model:
tfidfvect = TfidfVectorizer()

X = tfidfvect.fit_transform(df['tweet'])
y = df['age']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
nb = MultinomialNB()

nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[  249   427  1129     1     0]
 [    0  6347  3403     5     0]
 [    0   579 11464     7     0]
 [    0   388  3038  1270     0]
 [    0    61   190     0    12]]
              precision    recall  f1-score   support

       18-24       1.00      0.14      0.24      1806
       25-34       0.81      0.65      0.72      9755
       35-49       0.60      0.95      0.73     12050
       50-64       0.99      0.27      0.42      4696
       65-xx       1.00      0.05      0.09       263

    accuracy                           0.68     28570
   macro avg       0.88      0.41      0.44     28570
weighted avg       0.76      0.68      0.64     28570



In [15]:
# using a different tokenizer

tfidfvect = TfidfVectorizer(max_features=40000,
                            min_df=10,
                            ngram_range=(2,3), 
                            lowercase=False,
                            analyzer='char', 
                            binary=True,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

X = tfidfvect.fit_transform(df['tweet'])
y = df['age']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
nb = MultinomialNB()

nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[  558   595   643     8     2]
 [    7  6161  3429   143    15]
 [    7  1855 10015   161    12]
 [    0   660  2277  1756     3]
 [    0    23   163     5    72]]
              precision    recall  f1-score   support

       18-24       0.98      0.31      0.47      1806
       25-34       0.66      0.63      0.65      9755
       35-49       0.61      0.83      0.70     12050
       50-64       0.85      0.37      0.52      4696
       65-xx       0.69      0.27      0.39       263

    accuracy                           0.65     28570
   macro avg       0.76      0.48      0.55     28570
weighted avg       0.69      0.65      0.64     28570



In [36]:
# SGD
from sklearn.linear_model import SGDClassifier

tfidfvect = TfidfVectorizer(max_features=40000,
                            min_df=10,
                            ngram_range=(2,3), 
                            lowercase=False,
                            analyzer='char', 
                            binary=False,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

X = tfidfvect.fit_transform(df['tweet'])
y = df['age']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
clf = SGDClassifier()

clf.fit(X_train, y_train)

y_test_pred = clf.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[ 642  566  575   23    0]
 [  12 6331 3288  123    1]
 [   8 2118 9785  138    1]
 [   5  767 2015 1908    1]
 [   0   20   86    6  151]]
              precision    recall  f1-score   support

       18-24       0.96      0.36      0.52      1806
       25-34       0.65      0.65      0.65      9755
       35-49       0.62      0.81      0.70     12050
       50-64       0.87      0.41      0.55      4696
       65-xx       0.98      0.57      0.72       263

    accuracy                           0.66     28570
   macro avg       0.82      0.56      0.63     28570
weighted avg       0.70      0.66      0.65     28570



In [37]:
#logreg (best results so far)

tfidfvect = TfidfVectorizer(max_features=40000,
                            min_df=10,
                            ngram_range=(3,3), 
                            lowercase=False,
                            analyzer='char', 
                            binary=True,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

X = tfidfvect.fit_transform(df['tweet'])
y = df['age']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
logreg = LogisticRegression(solver='lbfgs')

logreg.fit(X_train, y_train)

y_test_pred = logreg.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[[ 790  503  487   26    0]
 [  21 7055 2531  148    0]
 [   7 1929 9907  207    0]
 [   3  706 1469 2518    0]
 [   0   27   83    7  146]]
              precision    recall  f1-score   support

       18-24       0.96      0.44      0.60      1806
       25-34       0.69      0.72      0.71      9755
       35-49       0.68      0.82      0.75     12050
       50-64       0.87      0.54      0.66      4696
       65-xx       1.00      0.56      0.71       263

    accuracy                           0.71     28570
   macro avg       0.84      0.61      0.69     28570
weighted avg       0.74      0.71      0.71     28570



In [40]:
#  logreg BOW model
tfidfvect = TfidfVectorizer(max_features=40000, 
                            lowercase=False,
                            analyzer='word', 
                            binary=False,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

X = tfidfvect.fit_transform(df['tweet'])
y = df['age']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
logreg = LogisticRegression(solver='lbfgs')

logreg.fit(X_train, y_train)

y_test_pred = logreg.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[[  768   497   515    26     0]
 [    7  7277  2321   150     0]
 [    6  1635 10211   198     0]
 [    0   666  1411  2619     0]
 [    1    35    82     2   143]]
              precision    recall  f1-score   support

       18-24       0.98      0.43      0.59      1806
       25-34       0.72      0.75      0.73      9755
       35-49       0.70      0.85      0.77     12050
       50-64       0.87      0.56      0.68      4696
       65-xx       1.00      0.54      0.70       263

    accuracy                           0.74     28570
   macro avg       0.86      0.62      0.70     28570
weighted avg       0.76      0.74      0.73     28570



In [31]:
# crossval - BOW
tfidfvect = TfidfVectorizer(max_features=40000, 
                            lowercase=False,
                            analyzer='word', 
                            binary=False,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

X = tfidfvect.fit_transform(df['tweet'])
y = df['age']

logreg = LogisticRegression()

#logreg.fit(X,y)

scores = cross_val_score(logreg, X, y, cv=10, scoring = 'accuracy')
print("Accuracy", scores)

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this war

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\

Accuracy [0.4355008  0.39910408 0.47305054 0.51469971 0.48680434 0.48445813
 0.4791375  0.4600252  0.46509837 0.4034449 ]


In [32]:
scores.mean()

0.46013235696354765

In [37]:
# crossval - ngrams
tfidfvect = TfidfVectorizer(#max_features=40000, 
                            #lowercase=False,
                            #analyzer='char', 
                            #ngram_range=(3,3),
                            #binary=False,
                            #tokenizer=word_tokenize,
                            #token_pattern='(?u)\\b\\w+\\b'
                            )

X = tfidfvect.fit_transform(df['tweet'])
y = df['age']

logreg = LogisticRegression()

#logreg.fit(X,y)

scores = cross_val_score(logreg, X, y, cv=5, scoring = 'accuracy')
print("Accuracy", scores)

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy [0.39960099 0.46718701 0.41666083 0.45440862 0.41085868]


In [38]:
scores.mean()

0.42974322853813235

In [219]:
# model:
tfidfvect = TfidfVectorizer(max_features=55000,
                            min_df=10,
                            ngram_range=(3,3), 
                            lowercase=False,
                            analyzer='char', 
                            binary=True,
                            token_pattern='(?u)\\b\\w+\\b')

In [220]:
tweets = df['tweet']
X = tfidfvect.fit_transform(tweets)
y = df['age']


In [221]:
#tfidfvect.get_feature_names()

In [222]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [223]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[ 587  340  373    9    0]
 [   8 5401 3030   58    9]
 [  17 1218 9691   68    5]
 [   2  504 2086 1450    6]
 [   0   36  156    2   76]]
              precision    recall  f1-score   support

       18-24       0.96      0.45      0.61      1309
       25-34       0.72      0.63      0.67      8506
       35-49       0.63      0.88      0.74     10999
       50-64       0.91      0.36      0.51      4048
       65-xx       0.79      0.28      0.42       270

    accuracy                           0.68     25132
   macro avg       0.80      0.52      0.59     25132
weighted avg       0.73      0.68      0.67     25132



In [17]:
lin = LinearSVC()
lin.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[  406   437   492     4     1]
 [   13  6070  3086    76     9]
 [    9  1826 10250   100     7]
 [    1   651  1933  1602    10]
 [    0    27   152     4   181]]
              precision    recall  f1-score   support

       18-24       0.95      0.30      0.46      1340
       25-34       0.67      0.66      0.66      9254
       35-49       0.64      0.84      0.73     12192
       50-64       0.90      0.38      0.54      4197
       65-xx       0.87      0.50      0.63       364

    accuracy                           0.68     27347
   macro avg       0.81      0.54      0.60     27347
weighted avg       0.71      0.68      0.66     27347



In [225]:
balanced_df = df.groupby('age')
balanced_df = pd.DataFrame(balanced_df.apply(lambda x: x.sample(balanced_df.size().min()).reset_index(drop=True)))

tfidfvect = TfidfVectorizer(max_features=6000, 
                            ngram_range=(3,3), 
                            lowercase=False,
                            analyzer='char', 
                            binary=False,
                            token_pattern='(?u)\\b\\w+\\b')

tweets = balanced_df['tweet']
X = tfidfvect.fit_transform(tweets)
y = balanced_df['age']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

nb = MultinomialNB()
nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[158  54  44  15   5]
 [ 10 123  54  35  16]
 [  5  90 112  39  14]
 [  6  55  55 131   8]
 [  2  31  23   8 182]]
              precision    recall  f1-score   support

       18-24       0.87      0.57      0.69       276
       25-34       0.35      0.52      0.42       238
       35-49       0.39      0.43      0.41       260
       50-64       0.57      0.51      0.54       255
       65-xx       0.81      0.74      0.77       246

    accuracy                           0.55      1275
   macro avg       0.60      0.55      0.57      1275
weighted avg       0.60      0.55      0.57      1275



## PAN15

In [179]:
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/PAN15train.csv')
df = df.drop('Unnamed: 0', 1)
df ['tweet'] = df['tweet'].replace(r'\t',  ' ', regex=True)
df['age'].value_counts()

18-24    5428
25-34    5283
35-49    1845
50-XX    1033
Name: age, dtype: int64

In [173]:
tfidfvect = TfidfVectorizer(max_features=6000, 
                            ngram_range=(3,3), 
                            lowercase=False,
                            analyzer='char', 
                            binary=False,
                            token_pattern='(?u)\\b\\w+\\b')
tfidfvect

TfidfVectorizer(analyzer='char', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=False, max_df=1.0, max_features=6000,
                min_df=1, ngram_range=(3, 3), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

In [120]:
X = tfidfvect.fit_transform(df['tweet'])
y = df['age']

In [121]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [122]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[1363  228    6    2]
 [ 532 1069   13    1]
 [ 134  266  150    1]
 [ 157  136    9   10]]
              precision    recall  f1-score   support

       18-24       0.62      0.85      0.72      1599
       25-34       0.63      0.66      0.65      1615
       35-49       0.84      0.27      0.41       551
       50-XX       0.71      0.03      0.06       312

    accuracy                           0.64      4077
   macro avg       0.70      0.45      0.46      4077
weighted avg       0.66      0.64      0.60      4077



In [186]:
#dropping 50-xx:

df = df[df['age'] != '50-xx']
print(df['age'].value_counts())

18-24    5428
25-34    5283
35-49    1845
50-XX    1033
Name: age, dtype: int64


In [ ]:
tfidfvect = TfidfVectorizer(max_features=6000, 
                            ngram_range=(3,3), 
                            lowercase=False,
                            analyzer='char', 
                            binary=False,
                            token_pattern='(?u)\\b\\w+\\b')

In [181]:
X = tfidfvect.fit_transform(df['tweet'])
y = df['age']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
nb = MultinomialNB()
nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[905 155   4   1]
 [344 702   9   1]
 [ 93 184 107   0]
 [105  97   6   5]]
              precision    recall  f1-score   support

       18-24       0.63      0.85      0.72      1065
       25-34       0.62      0.66      0.64      1056
       35-49       0.85      0.28      0.42       384
       50-XX       0.71      0.02      0.05       213

    accuracy                           0.63      2718
   macro avg       0.70      0.45      0.46      2718
weighted avg       0.66      0.63      0.59      2718



In [174]:
# balanced
balanced_df = df.groupby('age')
balanced_df = pd.DataFrame(balanced_df.apply(lambda x: x.sample(balanced_df.size().min()).reset_index(drop=True)))
tfidfvect = TfidfVectorizer(max_features=6000, 
                            ngram_range=(3,3), 
                            lowercase=False,
                            analyzer='char', 
                            binary=False,
                            token_pattern='(?u)\\b\\w+\\b')

In [175]:
X = tfidfvect.fit_transform(balanced_df['tweet'])
y = balanced_df['age']

In [176]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42)

In [177]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[236  18  16  57]
 [ 63 128  69  45]
 [ 43  38 174  51]
 [ 38  30  42 192]]
              precision    recall  f1-score   support

       18-24       0.62      0.72      0.67       327
       25-34       0.60      0.42      0.49       305
       35-49       0.58      0.57      0.57       306
       50-XX       0.56      0.64      0.59       302

    accuracy                           0.59      1240
   macro avg       0.59      0.59      0.58      1240
weighted avg       0.59      0.59      0.58      1240



## Celebrities

In [3]:
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/celebrities.csv', encoding='utf8')

labels = ['18-24','25-34','35-46','50-64','65-xx']
age_cats = []

for x in df['age']:

    if x in range(18,25):
        x = labels[0]
    elif x in range(25,35):
        x = labels[1]
    elif x in range(35,50):
        x = labels[2]
    elif x in range(50,66):
        x = labels[3]
    elif x > 65:
        x = labels[4]
    age_cats.append(x)
#df['age_cat'] = pd.Categorical(df['age_cat'])

age_cats = np.array(age_cats)
df['age_cat'] = age_cats

df = df.drop('age', 1)
df['age_cat'].value_counts()

35-46    273360
50-64    199181
25-34     97195
65-xx     37376
18-24      3851
Name: age_cat, dtype: int64

In [10]:
X = df['text']
y = df['age_cat']


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

y_test.value_counts()

35-46    54534
50-64    39961
25-34    19397
65-xx     7521
18-24      780
Name: age_cat, dtype: int64

In [ ]:
print(df)

In [6]:
tfidfvect = TfidfVectorizer(max_features = 50000, ngram_range = (3,3))

X = tfidfvect.fit_transform(df['text'])
y = df['age_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

logreg = LogisticRegression() 
logreg.fit(X_train, y_train)

y_test_pred = logreg.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[   14    48   649    69     0]
 [    0  3155 14251  1987     4]
 [    0  1447 45261  7798    28]
 [    0   534 24677 14719    31]
 [    0   102  5188  1715   516]]
              precision    recall  f1-score   support

       18-24       1.00      0.02      0.04       780
       25-34       0.60      0.16      0.26     19397
       35-46       0.50      0.83      0.63     54534
       50-64       0.56      0.37      0.44     39961
       65-xx       0.89      0.07      0.13      7521

    accuracy                           0.52    122193
   macro avg       0.71      0.29      0.30    122193
weighted avg       0.56      0.52      0.47    122193



In [7]:
y.value_counts(normalize=True)

35-46    0.447425
50-64    0.326012
25-34    0.159085
65-xx    0.061176
18-24    0.006303
Name: age_cat, dtype: float64

In [3]:
#base model

tfidfvect = TfidfVectorizer(max_features = 10000, ngram_range = (2,2))

X = tfidfvect.fit_transform(df['text'])
y = df['age_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

nb = MultinomialNB()
nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[    0    61   695    24     0]
 [    0  2792 14953  1650     2]
 [    0  1853 44476  8195    10]
 [    0   615 24807 14521    18]
 [    0   123  5158  2006   234]]


D:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

       18-24       0.00      0.00      0.00       780
       25-34       0.51      0.14      0.22     19397
       35-46       0.49      0.82      0.62     54534
       50-64       0.55      0.36      0.44     39961
       65-xx       0.89      0.03      0.06      7521

    accuracy                           0.51    122193
   macro avg       0.49      0.27      0.27    122193
weighted avg       0.54      0.51      0.46    122193



In [8]:
# model
tfidfvect = TfidfVectorizer(max_features=50000, 
                            ngram_range=(3,3), 
                            lowercase=False,
                            analyzer='char', 
                            binary=False,
                            token_pattern='(?u)\\b\\w+\\b')

In [9]:
X = tfidfvect.fit_transform(df['text'])
y = df['age_cat']

In [44]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [45]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[   17   170   570    23     0]
 [    0  4997 12423  1975     2]
 [    0  4000 38012 12519     3]
 [    0  1090 17149 21719     3]
 [    0   199  4086  3066   170]]
              precision    recall  f1-score   support

       18-24       1.00      0.02      0.04       780
       25-34       0.48      0.26      0.33     19397
       35-46       0.53      0.70      0.60     54534
       50-64       0.55      0.54      0.55     39961
       65-xx       0.96      0.02      0.04      7521

    accuracy                           0.53    122193
   macro avg       0.70      0.31      0.31    122193
weighted avg       0.56      0.53      0.50    122193



In [12]:
lsvc = LinearSVC()
lsvc.fit(X_train, y_train)

y_test_pred = lsvc.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[    0    53   677    50     0]
 [    0  2904 14310  2168    15]
 [    0  1776 43298  9432    28]
 [    0   629 23749 15524    59]
 [    0   140  4843  2220   318]]
              precision    recall  f1-score   support

       18-24       0.00      0.00      0.00       780
       25-34       0.53      0.15      0.23     19397
       35-46       0.50      0.79      0.61     54534
       50-64       0.53      0.39      0.45     39961
       65-xx       0.76      0.04      0.08      7521

    accuracy                           0.51    122193
   macro avg       0.46      0.27      0.27    122193
weighted avg       0.53      0.51      0.46    122193



In [11]:
lsvc = LinearSVC(penalty='l2')
lsvc.fit(X_train, y_train)

y_test_pred = lsvc.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[    0    53   677    50     0]
 [    0  2904 14310  2168    15]
 [    0  1777 43297  9432    28]
 [    0   629 23749 15524    59]
 [    0   140  4843  2220   318]]
              precision    recall  f1-score   support

       18-24       0.00      0.00      0.00       780
       25-34       0.53      0.15      0.23     19397
       35-46       0.50      0.79      0.61     54534
       50-64       0.53      0.39      0.45     39961
       65-xx       0.76      0.04      0.08      7521

    accuracy                           0.51    122193
   macro avg       0.46      0.27      0.27    122193
weighted avg       0.53      0.51      0.46    122193



In [44]:
#logreg - BOW
tfidfvect = TfidfVectorizer(max_features=40000,
                            min_df=5, 
                            lowercase=False,
                            analyzer='word', 
                            binary=False,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

X = tfidfvect.fit_transform(df['text'])
y = df['age_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
logreg = LogisticRegression(solver='lbfgs')

logreg.fit(X_train, y_train)

y_test_pred = logreg.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[[  102   114   468    95     1]
 [    7  7711  9561  2090    28]
 [    3  2136 43068  9239    88]
 [    1   593 13872 25404    91]
 [    0   124  3350  2581  1466]]
              precision    recall  f1-score   support

       18-24       0.90      0.13      0.23       780
       25-34       0.72      0.40      0.51     19397
       35-46       0.61      0.79      0.69     54534
       50-64       0.64      0.64      0.64     39961
       65-xx       0.88      0.19      0.32      7521

    accuracy                           0.64    122193
   macro avg       0.75      0.43      0.48    122193
weighted avg       0.66      0.64      0.62    122193



In [45]:
#logreg - char
tfidfvect = TfidfVectorizer(max_features=40000,
                            min_df=5, 
                            ngram_range=(3,3),
                            lowercase=False,
                            analyzer='char', 
                            binary=False,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

X = tfidfvect.fit_transform(df['text'])
y = df['age_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
logreg = LogisticRegression(solver='lbfgs')

logreg.fit(X_train, y_train)

y_test_pred = logreg.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\

[[  101   199   412    67     1]
 [    6  7951  9091  2329    20]
 [    4  2715 41371 10367    77]
 [    3   748 13442 25664   104]
 [    0   145  3406  2727  1243]]
              precision    recall  f1-score   support

       18-24       0.89      0.13      0.23       780
       25-34       0.68      0.41      0.51     19397
       35-46       0.61      0.76      0.68     54534
       50-64       0.62      0.64      0.63     39961
       65-xx       0.86      0.17      0.28      7521

    accuracy                           0.62    122193
   macro avg       0.73      0.42      0.46    122193
weighted avg       0.64      0.62      0.61    122193



In [53]:
#  low f-scores for 65-xx: drop age cat? 50 -xx instead?
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/celebrities.csv', encoding='utf8')

labels = ['18-24','25-34','35-49','50-xx']
age_cats = []

for x in df['age']:

    if x in range(18,25):
        x = labels[0]
    elif x in range(25,35):
        x = labels[1]
    elif x in range(35,50):
        x = labels[2]
    elif x > 49:
        x = labels[3]
    age_cats.append(x)
#df['age_cat'] = pd.Categorical(df['age_cat'])

age_cats = np.array(age_cats)
df['age_cat'] = age_cats

df = df.drop('age', 1)
df['age_cat'].value_counts()

35-49    273360
50-xx    236557
25-34     97195
18-24      3851
Name: age_cat, dtype: int64

In [3]:
tfidfvect = TfidfVectorizer(max_features=50000, 
                            ngram_range=(3,3), 
                            lowercase=False,
                            analyzer='char', 
                            binary=False,
                            token_pattern='(?u)\\b\\w+\\b')

In [159]:
X = tfidfvect.fit_transform(df['text'])
y = df['age_cat']

In [160]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [161]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[   39   219   472    50]
 [   23  6751  9579  3044]
 [   47  3736 34060 16691]
 [   10  1228 13975 32269]]
              precision    recall  f1-score   support

       18-24       0.33      0.05      0.09       780
       25-34       0.57      0.35      0.43     19397
       35-49       0.59      0.62      0.60     54534
       50-xx       0.62      0.68      0.65     47482

    accuracy                           0.60    122193
   macro avg       0.52      0.43      0.44    122193
weighted avg       0.59      0.60      0.59    122193



In [ ]:
# 0,6  0,44  0, 6


# add discriminating features for 18-24

In [54]:
#merged: logreg - BOW
tfidfvect = TfidfVectorizer(max_features=40000,
                            min_df=5, 
                            lowercase=False,
                            analyzer='word', 
                            binary=False,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

X = tfidfvect.fit_transform(df['text'])
y = df['age_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
logreg = LogisticRegression(solver='lbfgs')

logreg.fit(X_train, y_train)

y_test_pred = logreg.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


[[  102   114   445   119]
 [    7  7632  8967  2791]
 [    3  2064 40259 12208]
 [    1   639 13337 33505]]
              precision    recall  f1-score   support

       18-24       0.90      0.13      0.23       780
       25-34       0.73      0.39      0.51     19397
       35-49       0.64      0.74      0.69     54534
       50-xx       0.69      0.71      0.70     47482

    accuracy                           0.67    122193
   macro avg       0.74      0.49      0.53    122193
weighted avg       0.67      0.67      0.66    122193



In [ ]:
#logreg - ngrams
tfidfvect = TfidfVectorizer(max_features=40000,
                            ngram_range=(3,3),
                            min_df=5, 
                            lowercase=False,
                            analyzer='char', 
                            binary=False,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

X = tfidfvect.fit_transform(df['text'])
y = df['age_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)
logreg = LogisticRegression(solver='lbfgs')

logreg.fit(X_train, y_train)

y_test_pred = logreg.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

In [42]:
#balanced dataset:
balanced_df = df.groupby('age_cat')
balanced_df = pd.DataFrame(balanced_df.apply(lambda x: x.sample(balanced_df.size().min()).reset_index(drop=True)))

In [211]:
# base model:
tfidfvect = TfidfVectorizer()

X = tfidfvect.fit_transform(df['text'])
y = df['age_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

nb = MultinomialNB()
nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[    0     7   761    12     0]
 [    0  2479 15642  1276     0]
 [    0    83 49235  5216     0]
 [    0    14 18201 21745     1]
 [    0     2  5261  2161    97]]


D:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


              precision    recall  f1-score   support

       18-24       0.00      0.00      0.00       780
       25-34       0.96      0.13      0.23     19397
       35-46       0.55      0.90      0.69     54534
       50-64       0.72      0.54      0.62     39961
       65-xx       0.99      0.01      0.03      7521

    accuracy                           0.60    122193
   macro avg       0.64      0.32      0.31    122193
weighted avg       0.69      0.60      0.55    122193



In [133]:
#model
tfidfvect = TfidfVectorizer(max_features=15000, 
                            ngram_range=(2,3),
                            lowercase=False,
                            analyzer='char', 
                            binary=True,
                            token_pattern='(?u)\\b\\w+\\b')

In [134]:
X = tfidfvect.fit_transform(balanced_df['text'])
y = balanced_df['age_cat']

In [135]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

In [136]:
nb = MultinomialNB()
nb.fit(X_train, y_train)

y_test_pred = nb.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

[[546  81  18  29  92]
 [ 87 353 103  93 155]
 [ 46 183 135 167 254]
 [ 32  86  60 353 247]
 [ 22  78  57 154 420]]
              precision    recall  f1-score   support

       18-24       0.74      0.71      0.73       766
       25-34       0.45      0.45      0.45       791
       35-46       0.36      0.17      0.23       785
       50-64       0.44      0.45      0.45       778
       65-xx       0.36      0.57      0.44       731

    accuracy                           0.47      3851
   macro avg       0.47      0.47      0.46      3851
weighted avg       0.47      0.47      0.46      3851



In [43]:
tfidfvect = TfidfVectorizer(max_features=55000, 
                            ngram_range=(1,1),
                            lowercase=False,
                            analyzer='word', 
                            binary=True,
                            tokenizer=word_tokenize,
                            token_pattern='(?u)\\b\\w+\\b')

X = tfidfvect.fit_transform(balanced_df['text'])
y = balanced_df['age_cat']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state=42)

logreg = LogisticRegression()
logreg.fit(X, y)

y_test_pred = logreg.predict(X_test)

print(confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


[[706  22  17  11  10]
 [ 52 650  26  39  24]
 [ 58  49 598  48  32]
 [ 36  45  16 657  24]
 [ 43  25  21  31 611]]
              precision    recall  f1-score   support

       18-24       0.79      0.92      0.85       766
       25-34       0.82      0.82      0.82       791
       35-46       0.88      0.76      0.82       785
       50-64       0.84      0.84      0.84       778
       65-xx       0.87      0.84      0.85       731

    accuracy                           0.84      3851
   macro avg       0.84      0.84      0.84      3851
weighted avg       0.84      0.84      0.84      3851



In [51]:
tfidfvect = TfidfVectorizer(max_features=30000, 
                            ngram_range=(3,3),
                            lowercase=False,
                            analyzer='char', 
                            binary=False,
                            tokenizer=word_tokenize)

X = tfidfvect.fit_transform(balanced_df['text'])
y = balanced_df['age_cat']

logreg = LogisticRegression()

scores = cross_val_score(logreg, X, y, cv=10, scoring = 'accuracy')
print("Accuracy:", scores)
print ("Mean:", scores.mean())

D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
D:\Anaconda\lib\site-packages\sklearn\linear_model\logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Accuracy: [0.52901554 0.53194805 0.5361039  0.53922078 0.52571429 0.50857143
 0.53402597 0.52519481 0.50909091 0.50493506]
Mean: 0.5243820738846645


In [4]:
pipeline(tfidfvect, logreg).fit_transform(X, y)

NameError: name 'pipeline' is not defined

In [3]:
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/PAN16-train-preprocessed2.csv')

In [4]:
len(df)

20000

In [7]:
df.head()

,Unnamed: 0,Unnamed: 0.1,id,tweet,age,tokens,lemmas,pos,token_len,pos_len
0,10516,7565,0a9e35fd6f123137d585a482f2484d8e,"""£20m for Diego Costa?.................""",25-34,"['""', '£', '20', 'm', 'for', 'Diego', 'Costa',...","['""', '£', '20', 'm', 'for', 'Diego', 'Costa',...","['PUNCT', 'SYM', 'NUM', 'NUM', 'ADP', 'PROPN',...",9,9
1,10517,7566,0a9e35fd6f123137d585a482f2484d8e,"""The english media are going too love Diego Co...",25-34,"['""', 'The', 'english', 'media', 'are', 'going...","['""', 'the', 'english', 'media', 'be', 'go', '...","['PUNCT', 'DET', 'ADJ', 'NOUN', 'AUX', 'VERB',...",12,12
2,10518,7567,0a9e35fd6f123137d585a482f2484d8e,"""Join my league http://t.co/BZ8HmfHWhs""",25-34,"['""', 'Join', 'my', 'league', 'http://t.co/BZ8...","['""', 'Join', 'my', 'league', 'http://t.co/bz8...","['PUNCT', 'VERB', 'PRON', 'PROPN', 'X', 'PUNCT']",6,6
3,10519,7568,0a9e35fd6f123137d585a482f2484d8e,"""@BassTunedToRed @LFCTS should show games too!...",25-34,"['""', '@BassTuned', 'ToRed', '@LFCTS', 'should...","['""', '@basstuned', 'ToRed', '@lfct', 'should'...","['PUNCT', 'ADJ', 'ADJ', 'NOUN', 'AUX', 'VERB',...",15,15
4,10520,7569,0a9e35fd6f123137d585a482f2484d8e,"""@BassTunedToRed @LFCTS no an assist isnt a go...",25-34,"['""', '@BassTuned', 'ToRed', '@LFCTS', 'no', '...","['""', '@basstuned', 'ToRed', '@lfct', 'no', 'a...","['PUNCT', 'ADJ', 'INTJ', 'VERB', 'DET', 'DET',...",30,30


In [8]:
df = pd.read_csv('C:/Users/Jens/Notes Jupyter/Thesis-files/PAN16-train-preprocessed1.csv')

In [9]:
df.tail()

,Unnamed: 0,Unnamed: 0.1,id,tweet,age,tokens,lemmas,pos,token_len,pos_len
9995,10511,7560,0a9e35fd6f123137d585a482f2484d8e,"""how many people have actually seen this guy f...",25-34,"['""', 'how', 'many', 'people', 'have', 'actual...","['""', 'how', 'many', 'people', 'have', 'actual...","['PUNCT', 'ADV', 'ADJ', 'NOUN', 'AUX', 'ADV', ...",31,31
9996,10512,7561,0a9e35fd6f123137d585a482f2484d8e,"""theres on 1 DRS!!! Drag Reduction System! #F1...",25-34,"['""', 'theres', 'on', '1', 'DRS', '!!!', 'Drag...","['""', 'there', 'on', '1', 'DRS', '!!!', 'Drag'...","['PUNCT', 'VERB', 'ADP', 'NUM', 'NOUN', 'PUNCT...",17,17
9997,10513,7562,0a9e35fd6f123137d585a482f2484d8e,"""Roy Keane needs too come out of retirement NO...",25-34,"['""', 'Roy', 'Keane', 'needs', 'too', 'come', ...","['""', 'Roy', 'Keane', 'need', 'too', 'come', '...","['PUNCT', 'PROPN', 'PROPN', 'VERB', 'ADV', 'VE...",16,16
9998,10514,7563,0a9e35fd6f123137d585a482f2484d8e,"""@AdamDo_DaDay @SkySportsNews its his wife hah...",25-34,"['""', '@', 'AdamDo_DaDay', '@', 'SkySports', '...","['""', '@', 'adamdo_daday', '@', 'SkySports', '...","['PUNCT', 'ADP', 'PROPN', 'ADP', 'PROPN', 'NOU...",12,12
9999,10515,7564,0a9e35fd6f123137d585a482f2484d8e,"""@Green_Scouser i think he is shit, hope we do...",25-34,"['""', '@Green_Scouser', 'i', 'think', 'he', 'i...","['""', '@Green_Scouser', 'i', 'think', 'he', 'b...","['PUNCT', 'PROPN', 'PRON', 'VERB', 'PRON', 'AU...",16,16
